In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join('../../..')))

from Prototyping.setupConfig import setup_config
setup_config()

FUNCTIONS_WORKER_RUNTIME: python
pythonPath: C:\Users\Shubham Vishwakarma\AppData\Local\Microsoft\WindowsApps\python.exe
APPINSIGHTS_INSTRUMENTATIONKEY: f0cd5280-7f75-4c2e-9e92-93bc75b8a8c4
APPINSIGHTS_ConnectionString: InstrumentationKey=f0cd5280-7f75-4c2e-9e92-93bc75b8a8c4
KEY_VAULT_URI: https://kv-tradinglead.vault.azure.net/
TelemetryPropApp: TradingLead - Backend Function App
TelemetryPropEnv: local
TelemetryPropVersion: 1.0.0
TelemetryPropService: TradingLead - General Service
SCHEDULE_CRON_FOR_ACCESS_TOKEN_GENERATOR: 0 * * * * *
SCHEDULE_CRON_FOR_PARTICIPATION_DATA_STORE: 0 */10 * * * *
NumberOfDataParticipationDataToBeFetched: 20


In [2]:
from SharedCode.Repository.AccessToken.access_token import get_fyers_access_token
import json
import time
from SharedCode.Repository.KeyVault.keyvault_service import KeyVaultService
from SharedCode.Repository.Logger.logger_service import LoggerService
from SharedCode.Repository.Cache.redis_cache_service import RedisCacheService
from SharedCode.Utils.constants import Constants
import concurrent.futures
from SharedCode.Repository.Fyers.fyers_client_factory import FyersClientFactory
from SharedCode.Repository.Fyers.fyers_service import FyersService

kv_service = KeyVaultService()
telemetry = LoggerService()
redis_cache_service = RedisCacheService()

Failed to integrate module: opencensus.ext.logging.trace
No module named 'opencensus.ext.logging'


In [3]:
operation_id = "RandomOperationId"

tel_props = {
        Constants.SERVICE : Constants.access_token_generator_service,
        Constants.operation_id : operation_id,
    }

fyer_users_json = kv_service.get_secret("FyerUserDetails")
telemetry.info(fyer_users_json, tel_props)
fyer_users = json.loads(fyer_users_json)
results = []

In [5]:
fyer_users_json

'[   { "userId": "XS42465", "KvSecretName": "ShubhamFyersDetails" },   { "userId": "XJ13618", "KvSecretName": "JayantFyersDetails" } ]'

In [4]:
fyers_details_json = kv_service.get_secret(fyer_users[1]['KvSecretName'])
fyers_details = json.loads(fyers_details_json)

In [7]:
fyers_details

{'ClientId': 'EZ0JD38O15-100',
 'FyersPin': '1998',
 'FyersTotpSecretKey': 'JGN2I5KQXX2FXRZJ2PAT5WJJJDIQXAXC',
 'FyersUserName': 'XJ13618',
 'RedirectUri': 'https://trade.fyers.in/api-login/redirect-uri/index.html',
 'SecretKey': 'EDT0647NIQ',
 'ContactNumber': '8839183504'}

In [ ]:

client = FyersClientFactory.get_fyers_client(fyers_details)

In [7]:
tf="60"
ticker_name = "NSE:SBIN-EQ"

data = {
            "symbol":ticker_name,
            "resolution":tf,
            "date_format":"1",
            "range_from":"2023-10-10",
            "range_to":"2023-12-30",
            "cont_flag":"1"
        }

In [11]:
from datetime import datetime, timedelta
range_from = datetime.now() + timedelta(days = 2)
range_to = datetime.now() + timedelta(days = -4)

range_from = range_from.strftime('%Y-%m-%d')
range_to = range_to.strftime('%Y-%m-%d')

data = {
    "symbol":ticker_name,
    "resolution":tf,
    "date_format":"1",
    "range_from":range_from,
    "range_to":range_to,
    "cont_flag":"1"
}


In [5]:
fyerService = FyersService(fyers_details)

In [8]:
fyerService.get_close_data(ticker_name=ticker_name)

Api response : ok


[611.75,
 615.65,
 615.7,
 614,
 612.9,
 612.2,
 612.85,
 614.15,
 614,
 612.75,
 614.05,
 618.4,
 620.1,
 619,
 624.65,
 624.1,
 624.5,
 624.5,
 623.7,
 623.05,
 623.65,
 625.75,
 622.1,
 623.75,
 626.8,
 640.1,
 649.4,
 648.1]

In [ ]:
import datetime
from fyers_apiv3 import fyersModel
from SharedCode.Repository.Cache.redis_cache_service import RedisCacheService
from SharedCode.Utils.constants import Constants
from SharedCode.Utils.utility import FunctionUtils

In [ ]:
username = fyers_details[Constants.fyers_username]
client_id = fyers_details[Constants.client_id]
redis_key = FunctionUtils.get_key_for_user_access_token(username)
redis_cache_service = RedisCacheService()
access_token_for_user = redis_cache_service.get_decoded_value(redis_key)

In [ ]:
fyersClient = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token_for_user, log_path="")

In [ ]:
fyersClient.history(data=data)